In [2]:
import numpy as np
# tensorflow packages
import tensorflow as tf
!pip install tensorflow-hub
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                  split=('train[:60%]','train[60%:]','test'),
                                                  as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW6PG1M/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW6PG1M/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW6PG1M/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [6]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [7]:
# consists of the array of all the reviews
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [8]:
# positive reviews will be marked as 1 and the negative reviews will be marked as 0 or vice versa
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [9]:
embedding = "http://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape =[],dtype=tf.string, trainable = True)

In [10]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.7657859 , -3.882232  ,  3.913424  , -1.5557289 , -3.3362343 ,
        -1.7357956 , -1.9954445 ,  1.298955  ,  5.081597  , -1.1041285 ,
        -2.0503852 , -0.7267516 , -0.6567596 ,  0.24436145, -3.7208388 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489715 , -1.1315986 ],
       [ 1.8804485 , -2.5852385 ,  3.4066994 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.7855542 ,  1.3874227 ,  3.8476458 , -0.9256539 ,
        -1.896706  ,  1.2113281 ,  0.11474716,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015903 , -1.6390051 ],
       [ 0.71152216, -0.63532174,  1.7385626 , -1.1168287 , -0.54515934,
        -1.1808155 ,  0.09504453,  1.4653089 ,  0.66059506,  0.79308075,
        -2.2268343 ,  0.07446616, -1.4075902 , -0.706454  , -1.907037  ,
         1.4419788 ,  1.9551864 , -0.42660046, -2.8022065 ,  0.43727067]],
      dtype=float32)>

In [11]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer = 'adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics =["accuracy"])

In [16]:
history = model.fit(train_data.shuffle(10000).batch(100), epochs = 25,
                    validation_data = validation_data.batch(100), verbose=1)

Epoch 1/25
150/150 [==============================] - 45s 258ms/step - loss: 0.6867 - accuracy: 0.5989 - val_loss: 0.5768 - val_accuracy: 0.6734
Epoch 2/25
150/150 [==============================] - 33s 221ms/step - loss: 0.5140 - accuracy: 0.7312 - val_loss: 0.4665 - val_accuracy: 0.7625
Epoch 3/25
150/150 [==============================] - 24s 161ms/step - loss: 0.4061 - accuracy: 0.8107 - val_loss: 0.3959 - val_accuracy: 0.7993
Epoch 4/25
150/150 [==============================] - 23s 152ms/step - loss: 0.3337 - accuracy: 0.8527 - val_loss: 0.3540 - val_accuracy: 0.8274
Epoch 5/25
150/150 [==============================] - 17s 115ms/step - loss: 0.2826 - accuracy: 0.8789 - val_loss: 0.3265 - val_accuracy: 0.8534
Epoch 6/25
150/150 [==============================] - 17s 110ms/step - loss: 0.2421 - accuracy: 0.8998 - val_loss: 0.3109 - val_accuracy: 0.8576
Epoch 7/25
150/150 [==============================] - 14s 90ms/step - loss: 0.2102 - accuracy: 0.9181 - val_loss: 0.3097 - val_acc

In [19]:
results = model.evaluate(test_data.batch(100), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

250/250 - 2s - loss: 0.6293 - accuracy: 0.8401 - 2s/epoch - 10ms/step
loss: 0.629
accuracy: 0.840
